In [1]:
import networkx as nx
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
import NEMtropy as nem
import numpy as np

In [2]:
# Load the datasets from the assignment_1_data folder
path = "World_Trade_Web/"
files = [f for f in listdir(path) if isfile(join(path, f))]
gmls = [nx.read_graphml(path + f) for f in files]

graphs = {k: v for k, v in zip(files, gmls)}

In [3]:
# Test graph from the paper
# Do not execute for the assignment

# G=nx.DiGraph()
# G.add_nodes_from(["A","B","C","D","E","F","G","H"])
# G.add_edges_from([("A","B", {"weight": 10}),("A","E", {"weight":3}), ("C", "A", {"weight":1}), ("D", "A",{"weight":2}), ("B", "G",{"weight":5}), ("B", "F",{"weight":4}), ("H", "B",{"weight":6})])
# graphs={}
# graphs["test"] = G

In [4]:
def calc_strengths(G):
    in_ = {}
    out = {}
    for source, target, weight in G.edges(data="weight"):
        if source not in out:
            out[source] = 0
        if source not in in_:
            in_[source] = 0
        if target not in out:
            out[target] = 0
        if target not in in_:
            in_[target] = 0
        if weight is not None:
            out[source] += weight
            in_[target] += weight
    return {"in": in_, "out": out}

strengths = {}

for k, G in graphs.items():
    in_ = {}
    out = {}
    for source, target, weight in G.edges(data="weight"):
        if source not in out:
            out[source] = 0
        if source not in in_:
            in_[source] = 0
        if target not in out:
            out[target] = 0
        if target not in in_:
            in_[target] = 0
        if weight is not None:
            out[source] += weight
            in_[target] += weight
    strengths[k] = {"in": in_, "out": out}

In [5]:
W = {}
s_bar_sou_out = {}
s_bar_sou_in = {}
s_bar_tar_in = {}
s_bar_tar_out = {}
sigma_sou_out = {}
sigma_sou_in = {}
sigma_tar_in = {}
sigma_tar_out = {}

for k, strength in strengths.items():
    W[k] = sum(strength["out"].values())
    s_bar_sou_out[k] = 0
    s_bar_sou_in[k] = 0
    s_bar_tar_in[k] = 0
    s_bar_tar_out[k] = 0
    for node in strength["out"]:
        s_bar_sou_out[k] += strength["out"][node]*strength["out"][node]
        s_bar_sou_in[k] += strength["out"][node]*strength["in"][node]
        s_bar_tar_out[k] += strength["in"][node]*strength["out"][node]
        s_bar_tar_in[k] += strength["in"][node]*strength["in"][node]
    s_bar_sou_out[k] /= W[k]
    s_bar_sou_in[k] /= W[k]
    s_bar_tar_in[k] /= W[k]
    s_bar_tar_out[k] /= W[k]
    sigma_sou_out[k] = 0
    sigma_sou_in[k] = 0
    sigma_tar_in[k] = 0
    sigma_tar_out[k] = 0
    for node in strength["out"]:
        sigma_sou_out[k] += strength["out"][node]*(strength["out"][node] - s_bar_sou_out[k])**2
        sigma_sou_in[k] += strength["out"][node]*(strength["in"][node] - s_bar_sou_in[k])**2
        sigma_tar_in[k] += strength["in"][node]*(strength["in"][node] - s_bar_tar_in[k])**2
        sigma_tar_out[k] += strength["in"][node]*(strength["out"][node] - s_bar_tar_out[k])**2
    sigma_sou_out[k] = np.sqrt(sigma_sou_out[k]/W[k])
    sigma_sou_in[k] = np.sqrt(sigma_sou_in[k]/W[k])
    sigma_tar_in[k] = np.sqrt(sigma_tar_in[k]/W[k])
    sigma_tar_out[k] = np.sqrt(sigma_tar_out[k]/W[k])

In [6]:
rho_out_in = {}
rho_out_out = {}
rho_in_in = {}
rho_in_out = {}

for k, G in graphs.items():
    rho_out_in[k] = 0
    rho_out_out[k] = 0
    rho_in_in[k] = 0
    rho_in_out[k] = 0
    for source, target, weight in G.edges(data="weight"):
        if weight is not None:
            rho_out_in[k] += weight*(strengths[k]["out"][source] - s_bar_sou_out[k])*(strengths[k]["in"][target] - s_bar_tar_in[k])
            rho_out_out[k] += weight*(strengths[k]["out"][source] - s_bar_sou_out[k])*(strengths[k]["out"][target] - s_bar_tar_out[k])
            rho_in_in[k] += weight*(strengths[k]["in"][source] - s_bar_sou_in[k])*(strengths[k]["in"][target] - s_bar_tar_in[k])
            rho_in_out[k] += weight*(strengths[k]["in"][source] - s_bar_sou_in[k])*(strengths[k]["out"][target] - s_bar_tar_out[k])
    rho_out_in[k] /= W[k]*sigma_sou_out[k]*sigma_tar_in[k]
    rho_out_out[k] /= W[k]*sigma_sou_out[k]*sigma_tar_out[k]
    rho_in_in[k] /= W[k]*sigma_sou_in[k]*sigma_tar_in[k]
    rho_in_out[k] /= W[k]*sigma_sou_in[k]*sigma_tar_out[k]

    print(k + " | " + "rho out-in: " + str(rho_out_in[k]) + "| rho out-out: " + str(rho_out_out[k]) + "| rho in-in: " + str(rho_in_in[k]) + "| rho in-out: " + str(rho_in_out[k]))


WDN_1992.txt.graphml | rho out-in: -0.2664117215330637| rho out-out: -0.26076773855213525| rho in-in: -0.23043052971239772| rho in-out: -0.2178041687531879
WDN_1993.txt.graphml | rho out-in: -0.2332948861604653| rho out-out: -0.21690124032555627| rho in-in: -0.23436902905204038| rho in-out: -0.2021385793483107
WDN_1994.txt.graphml | rho out-in: -0.2280788632262324| rho out-out: -0.22662914473192905| rho in-in: -0.22585786853596368| rho in-out: -0.22015233241342597
WDN_1995.txt.graphml | rho out-in: -0.21154591029146982| rho out-out: -0.20812357271965518| rho in-in: -0.21449016601768575| rho in-out: -0.2074765749509863
WDN_1996.txt.graphml | rho out-in: -0.19698652263843464| rho out-out: -0.19471663098921604| rho in-in: -0.2022276276627051| rho in-out: -0.19651218763930592
WDN_1997.txt.graphml | rho out-in: -0.1918257224497967| rho out-out: -0.19315231407495667| rho in-in: -0.1932977484020739| rho in-out: -0.19017801772442888
WDN_1998.txt.graphml | rho out-in: -0.20399322491207095| rho 

In [22]:
# As an example we generate the adjacency matrix of a weighted undirected graph using the matrix_generator module of NEMtropy
adj_weigh = nem.matrix_generator.random_weighted_matrix_generator_uniform_custom_density(n=100,
                                                                                     p=0.4,
                                                                                     sym=True,
                                                                                     sup_ext=30,
                                                                                     intweights=True)

# Lets store the binary structure and strength sequence
degree_seq = adj_weigh.astype(bool).astype(float).sum(axis=1)
strength_seq = adj_weigh.sum(axis=1)

# Now we suppose that the weighted structure is not available
del adj_weigh

# We can initialiase our graph instance with degree and strength sequence
graph_weighted = nem.UndirectedGraph(degree_sequence=degree_seq, strength_sequence=strength_seq)

In [38]:
decms = {}
uecms = {}

for k, G in graphs.items():
    # Directed Enhanced Configuration Model
    out_degree = np.zeros(len(G))
    in_degree = np.zeros(len(G))
    n=0
    for node in G.nodes():
        out_degree[n] = G.out_degree(node)
        in_degree[n] = G.in_degree(node)
        n += 1
    out_strength = np.array(list(strengths[k]['out'].values()))
    in_strength = np.array(list(strengths[k]['in'].values()))

    # We can initialiase our graph instance with degree and strength sequence
    graph_weighted = nem.DirectedGraph(degree_sequence=np.concatenate([out_degree, in_degree]),
                                strength_sequence=np.concatenate([out_strength, in_strength]))
    graph_weighted.solve_tool(model="crema",
                          method="newton",
                          initial_guess="random",
                          adjacency="dcm_exp",
                          method_adjacency="newton")
    decms[k] = graph_weighted
    # Undirected Enhanced Configuration Model
    degree_seq = out_degree + in_degree
    strength_seq = out_strength + in_strength
    graph_weighted = nem.UndirectedGraph(degree_sequence=degree_seq, strength_sequence=strength_seq)
    graph_weighted.solve_tool(model="crema",
                          method="newton",
                          initial_guess="random",
                          adjacency="cm_exp",
                          method_adjacency="newton")
    uecms[k] = graph_weighted




solution error = 342170961427.0

solution error = 567206482101.9584

solution error = 361037925812.73413

solution error = 485445968154.2223

solution error = 406736885737.0

solution error = 796785496021.6816

solution error = 451894260718.09644

solution error = 903240604384.619

solution error = 471201956674.0554

solution error = 936112016929.749

solution error = 505506250174.78876

solution error = 1000046666707.0507

solution error = 536742356669.04285

solution error = 1035592514348.6445

solution error = 560403920746.8708

solution error = 1035771207256.8403

solution error = 590960843109.0779

solution error = 1083816634017.3927

solution error = 585263411933.8862

solution error = 1087625389336.7073

solution error = 592738877242.5737

solution error = 1059723347412.7062


In [39]:
for k, cm in decms.items():
    print(k + " | DECM | error degree " + str(cm.error_degree) + " | error strength " + str(cm.error_strength))

WDN_1992.txt.graphml | DECM | error degree 1.2056702303198108e-09 | error strength 342170961427.0
WDN_1993.txt.graphml | DECM | error degree 2.9074236351789295e-09 | error strength 361037925812.73413
WDN_1994.txt.graphml | DECM | error degree 5.046093320970613e-09 | error strength 406736885737.0
WDN_1995.txt.graphml | DECM | error degree 3.954315985765788e-09 | error strength 451894260718.09644
WDN_1996.txt.graphml | DECM | error degree 3.1559324042973458e-09 | error strength 471201956674.0554
WDN_1997.txt.graphml | DECM | error degree 2.9816646929248236e-09 | error strength 505506250174.78876
WDN_1998.txt.graphml | DECM | error degree 3.1343176942755235e-09 | error strength 536742356669.04285
WDN_1999.txt.graphml | DECM | error degree 4.04017441724136e-09 | error strength 560403920746.8708
WDN_2000.txt.graphml | DECM | error degree 3.6578065021330985e-09 | error strength 590960843109.0779
WDN_2001.txt.graphml | DECM | error degree 6.594138568516428e-09 | error strength 585263411933.88

In [40]:
for k, cm in uecms.items():
    print(k + " | UECM | error degree " + str(cm.error_degree) + " | error strength " + str(cm.error_strength))

WDN_1992.txt.graphml | UECM | error degree 50.0 | error strength 567206482101.9584
WDN_1993.txt.graphml | UECM | error degree 77.68637981967346 | error strength 485445968154.2223
WDN_1994.txt.graphml | UECM | error degree 101.96055580672638 | error strength 796785496021.6816
WDN_1995.txt.graphml | UECM | error degree 101.00283174035704 | error strength 903240604384.619
WDN_1996.txt.graphml | UECM | error degree 108.0 | error strength 936112016929.749
WDN_1997.txt.graphml | UECM | error degree 116.84963843719649 | error strength 1000046666707.0507
WDN_1998.txt.graphml | UECM | error degree 118.0 | error strength 1035592514348.6445
WDN_1999.txt.graphml | UECM | error degree 134.17497358066913 | error strength 1035771207256.8403
WDN_2000.txt.graphml | UECM | error degree 125.00000161524179 | error strength 1083816634017.3927
WDN_2001.txt.graphml | UECM | error degree 127.00004284421897 | error strength 1087625389336.7073
WDN_2002.txt.graphml | UECM | error degree 120.0 | error strength 10